In [36]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

#Step 1: Read and import the dataset

In [38]:
df = pd.read_csv('/content/drive/MyDrive/CSCE464/datasets/zoo.csv')
df

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize,class_type
0,aardvark,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
1,antelope,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1,1
2,bass,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0,4
3,bear,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1,1
4,boar,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1,1
97,wasp,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0,6
98,wolf,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1,1
99,worm,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,7


#Step 2: Apply one hot encoding to the class type column.

In [39]:
df['Mammal'] = df['class_type'].apply(lambda x:1 if x==1 else 0)
df['Bird'] = df['class_type'].apply(lambda x:1 if x==2 else 0)
df['Reptile'] = df['class_type'].apply(lambda x:1 if x==3 else 0)
df['Fish'] = df['class_type'].apply(lambda x:1 if x==4 else 0)
df['Amphibian'] = df['class_type'].apply(lambda x:1 if x==5 else 0)
df['Bug'] = df['class_type'].apply(lambda x:1 if x==6 else 0)
df[''] = df['class_type'].apply(lambda x:1 if x==7 else 0)

In [40]:
df

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,...,domestic,catsize,class_type,Mammal,Bird,Reptile,Fish,Amphibian,Bug,
0,aardvark,1,0,0,1,0,0,1,1,1,...,0,1,1,1,0,0,0,0,0,0
1,antelope,1,0,0,1,0,0,0,1,1,...,0,1,1,1,0,0,0,0,0,0
2,bass,0,0,1,0,0,1,1,1,1,...,0,0,4,0,0,0,1,0,0,0
3,bear,1,0,0,1,0,0,1,1,1,...,0,1,1,1,0,0,0,0,0,0
4,boar,1,0,0,1,0,0,1,1,1,...,0,1,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,wallaby,1,0,0,1,0,0,0,1,1,...,0,1,1,1,0,0,0,0,0,0
97,wasp,1,0,1,0,1,0,0,0,0,...,0,0,6,0,0,0,0,0,1,0
98,wolf,1,0,0,1,0,0,1,1,1,...,0,1,1,1,0,0,0,0,0,0
99,worm,0,0,1,0,0,0,0,0,0,...,0,0,7,0,0,0,0,0,0,1


#Step 3: Drop the animal_name and class_type columns

In [41]:
df = df.drop(['animal_name', 'class_type'], axis=1)
df

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,...,tail,domestic,catsize,Mammal,Bird,Reptile,Fish,Amphibian,Bug,
0,1,0,0,1,0,0,1,1,1,1,...,0,0,1,1,0,0,0,0,0,0
1,1,0,0,1,0,0,0,1,1,1,...,1,0,1,1,0,0,0,0,0,0
2,0,0,1,0,0,1,1,1,1,0,...,1,0,0,0,0,0,1,0,0,0
3,1,0,0,1,0,0,1,1,1,1,...,0,0,1,1,0,0,0,0,0,0
4,1,0,0,1,0,0,1,1,1,1,...,1,0,1,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,...,1,0,1,1,0,0,0,0,0,0
97,1,0,1,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
98,1,0,0,1,0,0,1,1,1,1,...,1,0,1,1,0,0,0,0,0,0
99,0,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,1


#Step 4 and 5: Split the dataset based on the 80/20 rule. Then reset index of each dataset (x_train, x_test, y_train, y_test)

In [42]:
x_data = df.iloc[:, :16]
y_data = df.iloc[:, 16:]

In [43]:
x_data

,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed,backbone,breathes,venomous,fins,legs,tail,domestic,catsize
0,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1
1,1,0,0,1,0,0,0,1,1,1,0,0,4,1,0,1
2,0,0,1,0,0,1,1,1,1,0,0,1,0,1,0,0
3,1,0,0,1,0,0,1,1,1,1,0,0,4,0,0,1
4,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,1,0,0,1,0,0,0,1,1,1,0,0,2,1,0,1
97,1,0,1,0,1,0,0,0,0,1,1,0,6,0,0,0
98,1,0,0,1,0,0,1,1,1,1,0,0,4,1,0,1
99,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


In [44]:
y_data

,Mammal,Bird,Reptile,Fish,Amphibian,Bug,
0,1,0,0,0,0,0,0
1,1,0,0,0,0,0,0
2,0,0,0,1,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...
96,1,0,0,0,0,0,0
97,0,0,0,0,0,1,0
98,1,0,0,0,0,0,0
99,0,0,0,0,0,0,1


In [45]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2)
x_train.reset_index(drop=True)
x_test.reset_index(drop=True)
y_train.reset_index(drop=True)
y_test.reset_index(drop=True)

,Mammal,Bird,Reptile,Fish,Amphibian,Bug,
0,0,0,0,0,0,0,1
1,0,0,0,1,0,0,0
2,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0
4,1,0,0,0,0,0,0
5,1,0,0,0,0,0,0
6,1,0,0,0,0,0,0
7,0,1,0,0,0,0,0
8,0,1,0,0,0,0,0
9,0,1,0,0,0,0,0


#Step 6: Normalize the dataset using the standard scaler

In [46]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

#Step 7: Build the TensorFlow and PyTorch softmax model.

In [47]:
#TensorFlow Softmax Model
X = tf.placeholder(tf.float32, [None, 16])
Y = tf.placeholder(tf.float32, [None, 7])
W = tf.Variable(tf.random_normal([16, 7]), name='weight')
b = tf.Variable(tf.random_normal([7]), name='bias')

tf_hypothesis = tf.nn.softmax(tf.matmul(X, W) + b)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(tf_hypothesis), axis=1))
optimizer = tf.train.GradientDescentOptimizer(learning_rate = 0.1).minimize(cost)

prediction = tf.argmax(tf_hypothesis, 1)
correct_prediction = tf.equal(tf.argmax(tf_hypothesis, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

#Model Training
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for epoch in range (1001):
  sess.run(optimizer, feed_dict={X: x_train, Y: y_train})

  if epoch % 100 == 0:
    _, acc = sess.run([cost, accuracy], feed_dict={X: x_test, Y: y_test})
    print("Epoch: {:5}\t Cost: {:.5f}\t ACC: {:.2%}".format(epoch, _, acc))

Epoch:     0	 Cost: 6.57434	 ACC: 4.76%
Epoch:   100	 Cost: 0.46777	 ACC: 85.71%
Epoch:   200	 Cost: 0.37360	 ACC: 90.48%
Epoch:   300	 Cost: 0.35939	 ACC: 90.48%
Epoch:   400	 Cost: 0.35815	 ACC: 90.48%
Epoch:   500	 Cost: 0.35885	 ACC: 90.48%
Epoch:   600	 Cost: 0.36025	 ACC: 90.48%
Epoch:   700	 Cost: 0.36198	 ACC: 90.48%
Epoch:   800	 Cost: 0.36386	 ACC: 90.48%
Epoch:   900	 Cost: 0.36581	 ACC: 90.48%
Epoch:  1000	 Cost: 0.36778	 ACC: 90.48%


In [48]:
#PyTorch softmax model

x_train, x_test = torch.FloatTensor(x_train), torch.FloatTensor(x_test)
y_train, y_test = torch.FloatTensor(y_train.to_numpy()), torch.FloatTensor(y_test.to_numpy())

class SoftmaxClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(16, 7)
    self.softmax = nn.Softmax()

  def forward(self, x):
    return self.softmax(self.linear(x))

In [49]:
model = SoftmaxClassifier()
optimizer = optim.SGD(model.parameters(), lr=0.1)

print("====Model Training====")
for epoch in range(1001):
  x = x_train
  y = y_train

  hypothesis = model(x)
  cost = F.cross_entropy(hypothesis, y)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 100 == 0:
    prediction = hypothesis
    correct_prediction = torch.argmax(prediction, 1) == torch.argmax(y_train, 1)
    torch_accuracy = correct_prediction.float().mean()

    print("Epoch {:4d} / {} Cost: {:.5f} Training ACC: {:.2f}%".format(epoch, 1000, cost.item(), torch_accuracy*100))

====Model Training====
Epoch    0 / 1000 Cost: 1.93662 Training ACC: 25.00%
Epoch  100 / 1000 Cost: 1.38736 Training ACC: 92.50%
Epoch  200 / 1000 Cost: 1.27904 Training ACC: 93.75%


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:1518: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


Epoch  300 / 1000 Cost: 1.24847 Training ACC: 98.75%
Epoch  400 / 1000 Cost: 1.22906 Training ACC: 98.75%
Epoch  500 / 1000 Cost: 1.21630 Training ACC: 98.75%
Epoch  600 / 1000 Cost: 1.20759 Training ACC: 100.00%
Epoch  700 / 1000 Cost: 1.20134 Training ACC: 100.00%
Epoch  800 / 1000 Cost: 1.19666 Training ACC: 100.00%
Epoch  900 / 1000 Cost: 1.19303 Training ACC: 100.00%
Epoch 1000 / 1000 Cost: 1.19015 Training ACC: 100.00%


#Step 8: Check and show the training and testing accuracies


In [50]:
print('====TensorFlow Testing====')
fish_like_data = np.array([[0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0]])
test_data = sess.run(tf_hypothesis, feed_dict={X: fish_like_data})
predicted_class = sess.run(tf.argmax(test_data, 1))
print("Predicted class probabilities:", test_data)
print("Predicted class:", predicted_class)
print("TensorFlow model accuracy: {:.2f} %".format(acc*100))

====TensorFlow Testing====
Predicted class probabilities: [[4.6124160e-02 1.5208138e-02 3.4830755e-01 2.7790162e-01 3.1221056e-01
  2.0253869e-04 4.5328932e-05]]
Predicted class: [2]
TensorFlow model accuracy: 90.48 %


In [53]:
print('====PyTorch Testing====')
test_data = torch.FloatTensor(fish_like_data)
predict_result = model(test_data)
print("Predicted class probabilities:", F.softmax(predict_result, dim=1))
print("Prediction: ", torch.argmax(F.softmax(predict_result, dim=1)))
print("PyTorch model accuracy: {:.2f} %".format(torch_accuracy.item()*100))

====PyTorch Testing====
Predicted class probabilities: tensor([[0.1364, 0.1334, 0.1333, 0.2061, 0.1451, 0.1234, 0.1223]],
       grad_fn=<SoftmaxBackward0>)
Prediction:  tensor(3)
PyTorch model accuracy: 100.00 %
